# Question 1

The first task involves taking input from the user and ensuring that the correct movie is queried form the data base.  This method will be the same for the script that queries the IMDB database in question 2.  

I choose seven misspelled movie title to test my scripted with. Note these are different from those used for the imdb database since the movies database contains fewer movies.

* "The poAtMan" - matches to "The Postman"

* "AVATor" - This one was used in class and matches "Avatar"

* "Mysery mes" - matches to "Mystery Men"

* "where harry met susan" - matches to "When Harry Met Sally..."

* "misses boubtfire" - matches to "Mrs. Doubtfire"

* "merica Beuty" - matches to "American Beauty"

* "frrest gmp" - matches to "Forest Gump"

#### Import statements

In [1]:
import psycopg2 as psy

#### Database Connection

In [2]:
conn = psy.connect('dbname=movies user=mtweed')

In [3]:
cur = conn.cursor()

#### Input
In order to standardize the input I usd string functions to ensure the capitalization was correct for the database.

In [4]:
query = input("Enter the name of a movie: ")

Enter the name of a movie: misses boubtfire


In [5]:
query = query.lower().title()

I found that I could be more selective of the results returned when matching a misspelled query if I omitted the word "the".

In [6]:
if "the" in query.lower().split():
    temp = query.lower().split()
    temp.remove('the')
    query = ' '.join(temp)
    query.title()

#### The SQL Matching Query

I used the similarity function for matching misspelled titles because it let me have much more control over how many results were returned as opposed to trigrams or ts_vectors. I found that a similarity value of 0.33 was optimal to return the desired values with as few other as possible.

In [7]:
cur.execute("""
            select distinct title
            from movies
            where similarity(title, %s) > 0.33;
            """, (query,))
results = cur.fetchall()
results = [results[x][0] for x in range(len(results))]

To further limit the number of title that are proposed for the user's selection, I used the levenshtein function to calculate the Levenshein numnber.  Again, I had to adjest the values to enure the the desired results were returned without too many undesired ones.  I set the score so that there was no penalty for replacements, a penalty on one for deletion, and a penalty of two for insertions.  These limits were found through trial and error. I set the cutoff threshold at 5 because, this values allowed for the addition of 'the' if necessary.

In [8]:
dist = []
for q in results:
    cur.execute("select levenshtein(%s, %s,2,1,0);", (q, query,))
    temp = cur.fetchall()
    dist.append(temp)

In [9]:
dist = [x[0][0] for x in dist]

In [12]:
pos = [results[x] for x in range(len(dist)) if dist[x] <= 5]

#### User Varification of Input

This script asks the user to select their intended query from a list. It shows ten possibilities at a time until the correct title is shown.

In [13]:
while True:
    opt = []
    if len(pos) == 0:
        print("Nothing found with that name.")
        break
        
    print("\nDid you mean:")
    if len(pos) > 10:
        for i in range(0,10):
            opt.append(pos.pop())
            print(str(i) + ") "+opt[i])
    else:
        for i in range(len(pos)):
            opt.append(pos.pop())
            print(str(i) + ") "+opt[i])
            
    sel = input("Make selection or press ENTER for more")
    
    if sel == '':
        continue
    else:
        sel = int(sel)
        
    if sel in range(0,10):
        query = opt[sel]
        break
    elif len(opt)==0:
        break
    else:
        continue


Did you mean:
0) Mrs. Doubtfire
Make selection or press ENTER for more0


## Finding Movies With Similar Genres

The movies database makes use of a cube object, so the first step I took was to query the database and characterize this object.

```
movies=# select title, genre from movies limit 5;
      title      |                          genre                          
-----------------+---------------------------------------------------------
 Star Wars       | (0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 10, 0, 0, 0)
 Forrest Gump    | (0, 0, 0, 5, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 American Beauty | (0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 Citizen Kane    | (0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 The Dark        | (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 5, 0)
(5 rows)
```

The cube is an 18 number tuple in which each digit indicated how representative that movies is of a particular genre.

```
movies=# select * from genres;
    name     | position 
-------------+----------
 Action      |        1
 Adventure   |        2
 Animation   |        3
 Comedy      |        4
 Crime       |        5
 Disaster    |        6
 Documentary |        7
 Drama       |        8
 Eastern     |        9
 Fantasy     |       10
 History     |       11
 Horror      |       12
 Musical     |       13
 Romance     |       14
 SciFi       |       15
 Sport       |       16
 Thriller    |       17
 Western     |       18
(18 rows)
```
Additionally, there is an extremely useful function (cube_distance) that allows the difference between two cube objects to be represented by a single number.

```
movies=> select i.title, j.title, cube_distance(i.genre, j.genre) dist from movies i, movies j where i.title = 'Star Wars' order by dist;
   title   |                       title                       |        dist        
-----------+---------------------------------------------------+--------------------
 Star Wars | Star Wars                                         |                  0
 Star Wars | Star Wars: Episode V - The Empire Strikes Back    |                  2
 Star Wars | Avatar                                            |                  5
 Star Wars | Explorers                                         |  5.744562646538029
 Star Wars | Krull                                             |   6.48074069840786
 Star Wars | The Lost World                                    |    6.6332495807108
 Star Wars | E.T. The Extra-Terrestrial                        |  7.615773105863909
 Star Wars | The People That Time Forgot                       |  8.831760866327848
 Star Wars | Fantastic Voyage                                  |  8.831760866327848
 Star Wars | Return from Witch Mountain                        |  8.831760866327848
 Star Wars | Voyage to the Bottom of the Sea                   |  8.831760866327848
 ```
 I used this function to create my query that would return movies of a similar genre as the user's query.


In [14]:
cur.execute("""
            select cube_distance(i.genre, j.genre), i.title, j.title 
            from movies i, movies j
            where i.title != j.title and i.title = %s
            """, [query])

In [15]:
dist = cur.fetchall()

In [16]:
dist.sort()

In [17]:
top10 = [x[2] for x in dist[0:10]]

In [18]:
print("\nThe top 10 movies most close in genre to {} are:".format(query))
for x in top10:
    print(x)


The top 10 movies most close in genre to Mrs. Doubtfire are:
Boogie Nights
Four Weddings And A Funeral
Forrest Gump
10
Addicted to Love
Alice's Restaurant
Arizona Dream
Arthur
August
Avanti!


In [19]:
conn.close()

## Example Console Output

The script for this question is available for testing at /usr/share/databases/Q1_movies.py

```
[mtweed@cs1 mtweed]$ ls
kbfinder.py  Q1_movies.py  Q2_imdb.py  Q3_fanfiction.py  scrub_stories.py
[mtweed@cs1 mtweed]$ chmod 755 Q1_movies.py 
[mtweed@cs1 mtweed]$ chmod 755 Q2_imdb.py 
[mtweed@cs1 mtweed]$ chmod 755 Q3_fanfiction.py 
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: The poAtMan

Did you mean:
0) Batman
1) The Postman
Make selection or press ENTER for more1

The top 10 movies most close in genre to The Postman are:
20.000 Leagues under the Sea
Black Beauty
Close Encounters of the Third Kind
E.T. The Extra-Terrestrial
Enemy Mine
Fahrenheit 451
Fantastic Voyage
Far and Away
First Men in the Moon
Fly Away Home
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: AVATor

Did you mean:
0) Avatar
Make selection or press ENTER for more0

The top 10 movies most close in genre to Avatar are:
Star Wars: Episode V - The Empire Strikes Back
Star Wars
Explorers
Krull
The Lost World
E.T. The Extra-Terrestrial
20.000 Leagues under the Sea
Barbarella
Biggles-Adventures in Time
Escape from the Planet of the Apes
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: Mysery mes

Did you mean:
0) Mystery Men
Make selection or press ENTER for more0

The top 10 movies most close in genre to Mystery Men are:
1941
A Day at the Races
A Little Bit of Soul
A Low Down Dirty Shame
A Midsummer Night's Sex Comedy
A Night at the Roxbury
A Private Function
Ace Ventura: Pet Detective
Air America
Alfie
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: where harry met susan

Did you mean:
0) When Harry Met Sally
Make selection or press ENTER for more0 

The top 10 movies most close in genre to When Harry Met Sally are:
10 Things I Hate About You
American Pie
As Good as It Gets
Clerks
Home Alone 2 - Lost in New York
Notting Hill
There's Something About Mary
1941
A Day at the Races
A Little Bit of Soul
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: misses boubtfire

Did you mean:
0) Mrs. Doubtfire
Make selection or press ENTER for more0

The top 10 movies most close in genre to Mrs. Doubtfire are:
Boogie Nights
Four Weddings And A Funeral
Forrest Gump
10
Addicted to Love
Alice's Restaurant
Arizona Dream
Arthur
August
Avanti!
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: merica Beuty

Did you mean:
0) American Beauty
Make selection or press ENTER for more0

The top 10 movies most close in genre to American Beauty are:
A Streetcar Named Desire
Boys Don't Cry
Dead Poets Society
The Bridges Of Madison County
The Green Mile
To kill a Mockingbird
William Shakespeare's Romeo + Juliet
10:30 P.M. Summer
54
A Civil Action
[mtweed@cs1 mtweed]$ ./Q1_movies.py 
Enter the name of a movie: frrest gmp

Did you mean:
0) Forrest Gump
Make selection or press ENTER for more0

The top 10 movies most close in genre to Forrest Gump are:
10
Addicted to Love
Alice's Restaurant
Arizona Dream
Arthur
August
Avanti!
Barefoot in the Park
Barfly
Beautiful Thing
```